# **Proyecto Final BD_NR**

> **¿Qué hace este programa?** Realiza una descarga de la API del Art Institute of Chicago de hasta 1000 obras para descargarla a MongoDB, y luego, usando una máquina virtual, prender una instancia y pasar los datos a Neo4j.

**INSTRUCCIONES** Corre la celda a continuación, llena los inputs deseados, y deja correr el resto de las celdas

## **1. Lectura de inputs**

In [1]:
from IPython.display import display
import ipywidgets as widgets

name_bd_mongo = widgets.Text(value='', description='BD:', layout=widgets.Layout(width='600px'))
name_col_mongo = widgets.Text(value='', description='Colección:', layout=widgets.Layout(width='600px'))
user_neo4j_box = widgets.Text(value='', description='User Neo4j:', layout=widgets.Layout(width='600px'))
password_neo4j_box = widgets.Text(value='', description='Password Neo4j:', layout=widgets.Layout(width='600px'))
local_box = widgets.Text(value='', description='Public IPv4 AWS:', layout=widgets.Layout(width='600px'))
submit_button = widgets.Button(description='Submit')

def submit_button_clicked(b):
    global bd_mongo, col_mongo, user_neo, password_neo, localhost
    bd_mongo = name_bd_mongo.value
    col_mongo = name_col_mongo.value
    user_neo = user_neo4j_box.value
    password_neo = password_neo4j_box.value
    localhost = local_box.value
    print("BD de Mongo:", bd_mongo)
    print("Colección de Mongo:", col_mongo)
    print("Usuario de Neo4j:", user_neo)
    print("Password de Neo4j:", password_neo)
    print("Public IPv4 de instancia:", localhost)

submit_button.on_click(submit_button_clicked)

input_widgets = widgets.VBox([name_bd_mongo, name_col_mongo,  user_neo4j_box, password_neo4j_box, local_box,submit_button])
display(input_widgets)
 

BD de Mongo: museito
Colección de Mongo: obra
Usuario de Neo4j: neo4j
Password de Neo4j: Putopacman14
Public IPv4 de instancia: 3.143.24.174


## **2. Import de bibliotecas**

In [2]:
from pprint import pprint
import requests
import pymongo
from neo4j import GraphDatabase

## **3. Conexión API a MongoDB**

Conexión a la API del Art Institute de Chicago para descargar 1000 obras de arte en una base de MongoDB.

**Descarga de obras de la API**

In [3]:
# Crear un arreglo para guardar las obras de arte

url = "https://api.artic.edu/api/v1/artworks?limit=100" # Descargar 500 obras
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print(f'Error en la descarga {response.status_code}')
    
# Descargar solo los datos
collection = data['data']

In [4]:
collection[1]

{'id': 74258,
 'api_model': 'artworks',
 'api_link': 'https://api.artic.edu/api/v1/artworks/74258',
 'is_boosted': False,
 'title': 'Colorado',
 'alt_titles': None,
 'thumbnail': {'lqip': '',
  'width': 3000,
  'height': 2029,
  'alt_text': 'A work made of gelatin silver print.'},
 'main_reference_number': '1989.210.3',
 'has_not_been_viewed_much': True,
 'boost_rank': None,
 'date_start': 1967,
 'date_end': 1967,
 'date_display': '1967',
 'date_qualifier_title': 'Made',
 'date_qualifier_id': 4,
 'artist_display': 'Lee Friedlander\nAmerican, born 1934',
 'place_of_origin': 'United States',
 'dimensions': 'Image/paper: 19.2 × 28.6 cm (7 9/16 × 11 5/16 in.); Paper: 28 × 35.4 cm (11 1/16 × 13 15/16 in.)',
 'dimensions_detail': [{'depth_cm': 0,
   'depth_in': 0,
   'width_cm': 28.6,
   'width_in': 11.2599,
   'height_cm': 19.2,
   'height_in': 7.5591,
   'diameter_cm': 0,
   'diameter_in': 0,
   'clarification': 'Image/paper'},
  {'depth_cm': 0,
   'depth_in': 0,
   'width_cm': 35.4,
   'w

**Revisar la estructura de cada JSON de una obra**

**Subir las obras a una nueva BD y colección en MongoDB**

In [5]:
# Crear un cliente de MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')

# Poner nombre a la BD
db = client[bd_mongo]

# Poner nombre a la conexión
my_collection = db[col_mongo]
insert_result = my_collection.insert_many(collection)

## **4. Queries de MongoDB**

En esta sección, incluimos tres queries avanzados para nuestra BD de Mongo.

**Query 1**

> Explicación: 

In [6]:
# pipeline del query

pipeline1 = [
    {
        '$group': {
            '_id': {
                'artist': '$artist_title'
            },
            'conteo_estilos': {
                '$addToSet': '$style_title'
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'artista': '$_id.artist',
            'conteo_estilos': { '$size': '$conteo_estilos' }
        }
    },
    {
        '$sort': {
            'conteo_estilos': -1,
        }
    },{
        '$limit': 5
    }
]

# Query

result1 = my_collection.aggregate(pipeline1)

Print de los resultados

In [7]:
for document in result1:
    pprint(document)


{'artista': 'Pablo Picasso', 'conteo_estilos': 2}
{'artista': 'Ancient Roman', 'conteo_estilos': 2}
{'artista': 'Willard Frederic Elms', 'conteo_estilos': 1}
{'artista': 'Artist unknown', 'conteo_estilos': 1}
{'artista': None, 'conteo_estilos': 1}


**Query 2**

> Explicación: por cada artista, enlistar todos los años de publicación de una obra.

In [8]:
pipeline2 = [
    {"$group": {"_id": "$artist_title", "fechas_display": {"$push": "$date_display"}}},
    {"$project": {"_id": 0, "artista": "$_id", "fechas_display": 1}},
    {"$sort": {"artista": 1}}
]

result2 = my_collection.aggregate(pipeline2)


Print de los resultados

In [9]:
for doc in result2:
    pprint(doc)

{'artista': None, 'fechas_display': ['c. 1940-50']}
{'artista': 'American China Manufactory', 'fechas_display': ['c. 1770-1772']}
{'artista': 'Ancient Etruscan', 'fechas_display': ['470-450 BCE']}
{'artista': 'Ancient Greek',
 'fechas_display': ['Geometric Period (800–700 BCE)',
                    'Geometric Period (800–700 BCE)',
                    'Geometric Period (800–700 BCE)',
                    'Geometric Period (800–700 BCE)',
                    'Geometric Period (800–700 BCE)',
                    'Geometric Period (800–700 BCE)',
                    'Geometric Period (800–600 BCE)',
                    'Geometric Period (800–700 BCE)',
                    'Geometric Period (800–700 BCE)',
                    'Geometric Period (800–600 BCE)',
                    'Geometric Period (800–600 BCE)',
                    'Geometric Period (800–600 BCE)',
                    'Geometric Period (800–600 BCE)',
                    'Geometric Period (800–700 BCE)',
                  

**Query 3**

> Explicación: ordenar de menor a mayor, el promedio del ancho de las pinturas de cada artista

In [10]:
pipeline3 = [
    {
        '$group': {
            '_id': {
                'artista': '$artist_title'
            },
            'promedio_tamaños': {
                '$avg': '$thumbnail.width'
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'artista': '$_id.artista',
            'promedio_tamaño': '$promedio_tamaños'
        }
    },
    {
        '$sort': {
            'promedio_tamaño': 1,
        }
    }
]

result3 = my_collection.aggregate(pipeline3)


In [11]:
for i in result3:
    pprint(i)

{'artista': 'Lawrence N. Shustak', 'promedio_tamaño': None}
{'artista': 'Nathan Lerner', 'promedio_tamaño': 1024.0}
{'artista': 'Radcliffe Bailey', 'promedio_tamaño': 1627.0}
{'artista': 'Rogier van der Weyden', 'promedio_tamaño': 1644.0}
{'artista': 'Ancient Etruscan', 'promedio_tamaño': 1758.0}
{'artista': 'Thomas Wilmer Dewing', 'promedio_tamaño': 1864.0}
{'artista': 'Jasper Johns', 'promedio_tamaño': 2028.0}
{'artista': 'Jerzy Lewczyński', 'promedio_tamaño': 2056.0}
{'artista': 'Ancient Greek', 'promedio_tamaño': 2192.431818181818}
{'artista': 'Ibrahim El-Salahi', 'promedio_tamaño': 2252.0}
{'artista': 'Lucas Cranach, the Elder', 'promedio_tamaño': 2260.0}
{'artista': 'Ancient Roman', 'promedio_tamaño': 2344.5}
{'artista': None, 'promedio_tamaño': 2634.0}
{'artista': 'Nasca', 'promedio_tamaño': 2833.6666666666665}
{'artista': 'Yasuhiro Ishimoto', 'promedio_tamaño': 2953.0}
{'artista': 'Artist unknown', 'promedio_tamaño': 2987.0}
{'artista': 'Lee Friedlander', 'promedio_tamaño': 300

## **5. Conexión de MongoDB a Neo4j**

Conexión de los datos de MongoDB a Neo4j, para así poder crear una nueva base de grafos, relacionando obras de arte entre ellas.

Clase para graficar pinturas

In [12]:
# Conectarse a Neo4j
class GrafoPinturas:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def crea_pintura(self, ID, nombre, lugar, artista, estilo, tipo):
        try:
            consulta = """
            CREATE (n:Pintura {id:$ID,
                               nombre:$nombre, 
                               lugar:$lugar,
                               artista:$artista,
                               estilo:$estilo,
                               tipo:$tipo})
            """
            summary = self.driver.execute_query(
                consulta,
                ID=ID,
                nombre=nombre,
                lugar=lugar,
                artista=artista,
                estilo=estilo,
                tipo=tipo,
                database_='neo4j' 
            ).summary

            #Imprimimos el resumen para asegurar que se cree correctamente
            print("Created {nodes_created} nodes in {time} ms.".format(
                nodes_created=summary.counters.nodes_created,
                time=summary.result_available_after
            ))
        except Exception as e:
            print("An exception occurred")
        
        
    def crea_artista(self, ID, nombre):
        try:
            consulta = """
            CREATE (n:Artista {id:$ID,
                               nombre:$nombre})
            """
            summary = self.driver.execute_query(
                consulta,
                ID=ID,
                nombre=nombre,
                database_='neo4j'  
            ).summary

            #Imprimimos el resumen para asegurar que se cree correctamente
            print("Created {nodes_created} nodes in {time} ms.".format(
                nodes_created=summary.counters.nodes_created,
                time=summary.result_available_after
            ))
        except Exception as e:
            print("An exception occurred")
     
    
    def crea_estilo(self, ID, nombre):
        try:
            consulta = """
            CREATE (n:Estilo {id:$ID,
                               nombre:$nombre})
            """
            summary = self.driver.execute_query(
                consulta,
                ID=ID,
                nombre=nombre,
                database_='neo4j'  
            ).summary

            #Imprimimos el resumen para asegurar que se cree correctamente
            print("Created {nodes_created} nodes in {time} ms.".format(
                nodes_created=summary.counters.nodes_created,
                time=summary.result_available_after
            ))
        except Exception as e:
            print("An exception occurred")
            print(e)
            

    def crea_lugar(self, ID, nombre):
        try:
            consulta = """
            CREATE (n:Pais {id:$ID,
                               nombre:$nombre})
            """
            summary = self.driver.execute_query(
                consulta,
                ID=ID,
                nombre=nombre,
                database_='neo4j'  
            ).summary

            #Imprimimos el resumen para asegurar que se cree correctamente
            print("Created {nodes_created} nodes in {time} ms.".format(
                nodes_created=summary.counters.nodes_created,
                time=summary.result_available_after
            ))
        except Exception as e:
            print("An exception occurred")
            
            
    def relacion(self, consulta):
        try:
            records, summary, keys = self.driver.execute_query(consulta,database_='neo4j')
            print(f"Query counters: {summary.counters}.")
        except:
            print("An exception occurred")
            
            
    def consulta(self, consulta):
        records, summary, keys = self.driver.execute_query(
            consulta,
            database_="neo4j",
        )
        return records, summary, keys


Conexión

In [13]:
# Conectarse a Neo4j

link = 'bolt://' + localhost + ':7687'

conexion_grafo_pinturas = GrafoPinturas(link, user_neo, password_neo) # instancia de la clase

### **4.1. Conseguir entidades**

In [14]:
# Artistas
pipeline_artists = [
    {"$group": {"_id": "$artist_title"}},
    {"$project": {"_id": 0, "artista": "$_id"}},
    {"$sort": {"artista": 1}}
]

artistas = my_collection.aggregate(pipeline_artists)

# Obras

pipeline_obras = [{ "$project": {
      "nombre": "$title",
      "id": "$id",
      "lugar_origen": "$place_of_origin",
      "artista": "$artist_title",
      "estilo": "$style_title",
      "tipo": "$medium_display",
      "_id": 0
    }
  }
]


obras = my_collection.aggregate(pipeline_obras)


# Lugares

pipeline_lugares = [
  {"$group": {"_id": "$place_of_origin"}},
  {"$project":{"_id":0, "lugar":"$_id"}},
  {"$sort":{"lugar":1}}
]


lugares = my_collection.aggregate(pipeline_lugares)


# Estilo

pipeline_estilos = [
  {"$group": {"_id": "$style_title"}},
  {"$project":{"_id":0, "estilo":"$_id"}},
  {"$sort":{"estilo":1}}
]


estilos = my_collection.aggregate(pipeline_estilos)


### **4.2 Creamos los nodos en Neo4j**

In [15]:
for pintura in obras:
    conexion_grafo_pinturas.crea_pintura(
        ID=pintura['id'], 
        nombre=pintura['nombre'], 
        lugar=pintura['lugar_origen'], 
        artista=pintura['artista'], 
        estilo=pintura['estilo'], 
        tipo=pintura['tipo']
    )
    
for contador, artista in enumerate(artistas):
    conexion_grafo_pinturas.crea_artista(
        ID=contador,
        nombre =artista['artista']
    )
    
    
for contador, estilo in enumerate(estilos):
    conexion_grafo_pinturas.crea_estilo(
        ID=contador,
        nombre =estilo['estilo']
    )
    
for contador, lugar in enumerate(lugares):
    conexion_grafo_pinturas.crea_lugar(
        ID=contador,
        nombre =lugar['lugar']
    ) 
    

Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 2 ms.
Created 1 nodes in 16 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 5 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 15 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 3 ms.
Created 1 nodes in 3 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 2 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 0 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms.
Created 1 nodes in 1 ms

### **4.3. Creamos las relaciones**

In [16]:
relacion1 = """
MATCH (p:Pintura),(a:Artista)
WHERE p.artista = a.nombre
CREATE (p)-[:PINTADA_POR]->(a);
"""
relacion2 = """
MATCH (p:Pintura),(c:Pais)
WHERE p.lugar = c.nombre
CREATE (p)-[:ES_DE]->(c);
"""
relacion3 = """
MATCH (p:Pintura),(e:Estilo)
WHERE p.estilo = e.nombre
CREATE (p)-[:TIENE]->(e);
"""
conexion_grafo_pinturas.relacion(relacion1)
conexion_grafo_pinturas.relacion(relacion2)
conexion_grafo_pinturas.relacion(relacion3)

Query counters: {'relationships_created': 99}.
Query counters: {'relationships_created': 97}.
Query counters: {'relationships_created': 31}.


### **6. Queries en Neo4j**

**Query 1**

> Explicación: te regresa todas las pinturas de Picasso incluyendo una visualización del grafo (si lo corres en la interfaz)

In [17]:
consulta1 = """
MATCH (o:Pintura)-[rel]->(e:Artista)
WHERE e.nombre CONTAINS 'Pablo Picasso'
RETURN o, rel, e;
"""
records1, summary1, keys1 = conexion_grafo_pinturas.consulta(consulta1)

for record in records1:
    print(record.data())

{'o': {'estilo': 'reclining', 'tipo': 'Oil on canvas', 'artista': 'Pablo Picasso', 'lugar': 'Spain', 'id': 23968, 'nombre': 'Nude under a Pine Tree'}, 'rel': ({'estilo': 'reclining', 'tipo': 'Oil on canvas', 'artista': 'Pablo Picasso', 'lugar': 'Spain', 'id': 23968, 'nombre': 'Nude under a Pine Tree'}, 'PINTADA_POR', {'nombre': 'Pablo Picasso', 'id': 25}), 'e': {'nombre': 'Pablo Picasso', 'id': 25}}
{'o': {'estilo': 'Cubism', 'tipo': 'Oil on canvas', 'artista': 'Pablo Picasso', 'lugar': 'Spain', 'id': 84239, 'nombre': 'Portrait of Sylvette David'}, 'rel': ({'estilo': 'Cubism', 'tipo': 'Oil on canvas', 'artista': 'Pablo Picasso', 'lugar': 'Spain', 'id': 84239, 'nombre': 'Portrait of Sylvette David'}, 'PINTADA_POR', {'nombre': 'Pablo Picasso', 'id': 25}), 'e': {'nombre': 'Pablo Picasso', 'id': 25}}
{'o': {'tipo': 'White chalk on plywood', 'estilo': 'Cubism', 'artista': 'Pablo Picasso', 'lugar': 'Spain', 'id': 28019, 'nombre': 'Richard J. Daley Center Sculpture'}, 'rel': ({'tipo': 'White 

**Query 2**

> Explicación: te regresa los tipos de obra de arte que más están relacionados a obras junto con el número de obras.

In [18]:
consulta2 = """
MATCH (e:Pintura) return e.tipo, count(*) as c order by c desc;
"""
records2, summary2, keys2 = conexion_grafo_pinturas.consulta(consulta2)

for record in records2:
    print(record.data())

{'e.tipo': 'Bronze', 'c': 46}
{'e.tipo': 'Gelatin silver print', 'c': 14}
{'e.tipo': 'Ceramic and pigment', 'c': 6}
{'e.tipo': 'Oil on canvas', 'c': 5}
{'e.tipo': 'Mixed media on cream wove paper', 'c': 3}
{'e.tipo': 'Pencil and crayon on off-white paper', 'c': 3}
{'e.tipo': 'Woodcut in black on ivory laid paper', 'c': 2}
{'e.tipo': 'Gelatin silver print, from the portfolio "Black Jews"', 'c': 1}
{'e.tipo': 'Marble', 'c': 1}
{'e.tipo': 'Transparent and opaque water-based pigments on paper', 'c': 1}
{'e.tipo': 'Bronze, silver, and copper', 'c': 1}
{'e.tipo': 'Lithograph from one stone in gray on cream wove paper', 'c': 1}
{'e.tipo': 'Color lithograph on white Japan paper', 'c': 1}
{'e.tipo': 'Porcelain', 'c': 1}
{'e.tipo': 'Soft-paste porcelain, underglaze blue decoration', 'c': 1}
{'e.tipo': 'Mahogany, satinwood, and brass', 'c': 1}
{'e.tipo': 'Favrile glass, mirrored glass, and bronze', 'c': 1}
{'e.tipo': 'Pencil and gouache on paper', 'c': 1}
{'e.tipo': 'Pencil, crayon and watercolor

**Query 3**

> Explicación: te regresa todos los artistas más importantes usando GDS.PageRank


Ejecutar en la interfaz:

> CALL gds.graph.create('pinturasYArtistas', ['Artista', 'Pintura'], ['PINTADA_POR']) YIELD graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipCount AS rels;


Luego, ejecturar:

> CALL gds.pageRank.stream('pinturasYArtistas') YIELD nodeId, score RETURN gds.util.asNode(nodeId).nombre AS nombre, score ORDER BY score DESC, nombre ASC


## **7. Fin conexión de Neo4j**

In [ ]:
conexion_grafo_pinturas.close()